In [1]:
import datetime
TIMESTAMP = format(f"{datetime.datetime.now():%Y_%m_%d_%H_%M}")
print(TIMESTAMP)

2020_09_27_08_46


In [1]:
TIMESTAMP = '2019_12_19_16_41'
ENV='high'
dataset='gst'

In [2]:
import os
if not os.path.exists('./data/{}/{}'.format(dataset, TIMESTAMP)):
    os.makedirs('./data/{}/{}'.format(dataset, TIMESTAMP), mode=0o777)
if not os.path.exists('./data/{}/{}/logs'.format(dataset, TIMESTAMP)):
    os.makedirs('./data/{}/{}/logs'.format(dataset, TIMESTAMP), mode=0o777)
if not os.path.exists('./data/{}/{}/best_models'.format(dataset, TIMESTAMP)):
    os.makedirs('./data/{}/{}/best_models'.format(dataset, TIMESTAMP), mode=0o777)
if not os.path.exists('./data/{}/{}/graph'.format(dataset, TIMESTAMP)):
    os.makedirs('./data/{}/{}/graph'.format(dataset, TIMESTAMP), mode=0o777)
if not os.path.exists('./data/{}/{}/filters'.format(dataset, TIMESTAMP)):
    os.makedirs('./data/{}/{}/filters'.format(dataset, TIMESTAMP), mode=0o777)
if not os.path.exists('./data/{}/{}/cluster_analysis'.format(dataset, TIMESTAMP)):
    os.makedirs('./data/{}/{}/cluster_analysis'.format(dataset, TIMESTAMP), mode=0o777)

In [3]:
import utils_tf
import utils
from Bio import SeqIO

In [4]:
all_seqs = utils.load_pickle('./data/{}/{}_{}.pickle'.format(dataset,dataset,ENV))


In [5]:
max_seq_size = 0
for _id, seq_data in all_seqs.items():
    max_seq_size = max(max_seq_size,seq_data['sequence'][0].__len__())

In [6]:
_train_dataset = []
_train_labels = []
_train_ids = []

_test_dataset = []
_test_labels = []
_test_ids = []
hsa_train_id, hsa_test_id = utils.cluster_res_to_dataset('./data/{}/{}_{}.results.clstr'.format(dataset,dataset,ENV))
LOW_AFFINITY=0
HIGH_AFFINITY=2
for _id, seq_data in all_seqs.items():
    lab = int(seq_data['labels'][0])
    if _id in hsa_train_id:
        # in general
        # _train_labels.append(lab)
        # _train_dataset.append(utils_tf.to_one_hot_prot(seq_data['sequence'][0], max_seq_size))
        # _train_ids.append(_id)
        
        # compare high vs low
        if LOW_AFFINITY == lab:
            _train_labels.append(lab)
            _train_dataset.append(utils_tf.to_one_hot_prot(seq_data['sequence'][0], max_seq_size))
            _train_ids.append(_id)
        elif HIGH_AFFINITY == lab:
            _train_labels.append(1)
            _train_dataset.append(utils_tf.to_one_hot_prot(seq_data['sequence'][0], max_seq_size))
            _train_ids.append(_id)
    else:
        # _test_labels.append(lab)
        # _test_dataset.append(utils_tf.to_one_hot_prot(seq_data['sequence'][0], max_seq_size))
        # _test_ids.append(_id)

        # compare high vs low
        if LOW_AFFINITY == lab:
            _test_labels.append(lab)
            _test_dataset.append(utils_tf.to_one_hot_prot(seq_data['sequence'][0], max_seq_size))
            _test_ids.append(_id)
        elif HIGH_AFFINITY == lab:
            _test_labels.append(1)
            _test_dataset.append(utils_tf.to_one_hot_prot(seq_data['sequence'][0], max_seq_size))
            _test_ids.append(_id)
    

In [7]:
utils.to_npz('./data/{}/{}/train_{}.npz'.format(dataset, TIMESTAMP, ENV), _train_dataset, _train_labels)
utils.to_npz('./data/{}/{}/test_{}.npz'.format(dataset, TIMESTAMP,ENV), _test_dataset, _test_labels)

utils.to_pickle(_train_ids, './data/{}/{}/train_ids_{}.pickle'.format(dataset, TIMESTAMP,ENV))
utils.to_pickle(_test_ids, './data/{}/{}/test_ids_{}.pickle'.format(dataset, TIMESTAMP, ENV))

Train model then analyse. Run the following command: train.py ...

In [8]:
RUN = 'K5Z523NP'
filter_length=7
input_folder = './data/{}/{}'.format(dataset,TIMESTAMP)
model = './data/{}/{}/best_models/{}/DeepNano_{}_{}.h5'.format(dataset, TIMESTAMP, RUN,ENV, RUN)
test = './data/{}/{}/test_{}.npz'.format(dataset,TIMESTAMP, ENV)
all_data_ids_fn = './data/{}/all_data_ids_{}.pickle'.format(dataset, ENV)

In [9]:
test_data, test_labels, classifier, layer_outputs, activations = utils_tf.predict(model, test)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [10]:
import model_analysis
import numpy as np
from os import path
import matplotlib.pyplot as plt
import utils_jer

findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.


In [11]:
filters, biases = classifier.layers[0].get_weights()
f_min, f_max = filters.min(), filters.max()
filters = (filters) / (f_max - f_min)
filters_mask = np.round(filters, decimals=1)
filters[filters_mask == 0] = 0
for filt in range(filters.shape[2]):
    filter_to_show_jer = np.zeros((filter_length,21))
    filter_to_show_jer[:,0:20] = filters[:,:,filt]
    fig = utils_jer.sequence_logo(filter_to_show_jer,figsize=(filter_length,filter_length), title="Filter num: {}".format(filt), show=False, data_type='weights');
    fig.savefig(path.join(input_folder, './filters/{}_{}_{}_filt{}.svg'.format(dataset, ENV, RUN, filt)));
    plt.close(fig)

In [12]:
model_analysis.model_analysis(
        test,
        input_folder,
        model,
        all_data_ids_fn,
        ENV,
        './data/{}/{}/test_ids_{}.pickle'.format(dataset,TIMESTAMP,ENV),
        '{}_{}'.format(dataset,RUN),
        [0,1],
        97
    )

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
